In [ ]:
import pandas as pd

input_file = '../assets/data/recommendation_dataset.csv'

dfp = pd.read_csv(input_file, sep=';', skiprows=1)
dfp = dfp.iloc[:, 1:].replace(99.0, None)

print(dfp)
mean = dfp.mean(axis=None, skipna=True)
dfp = dfp.fillna(mean)
print(dfp)

      -7.82  8.79 -9.66 -8.16 -7.52  -8.5 -9.85  4.17 -8.98 -4.76  ... 2.82.2  \
0      None -0.29  6.36  4.37 -2.38 -9.66 -0.73 -5.34  8.88  None  ...   2.82   
1      None  None  None  None  9.03  9.27  9.03  9.27  None  None  ...   None   
2      None  8.35  None  None   1.8  8.16 -2.82  6.21  None  1.84  ...   None   
3       8.5  4.61 -4.17 -5.39  1.36   1.6  7.04  4.61 -0.44  5.73  ...   5.19   
4     -6.17 -3.54  0.44  -8.5 -7.09 -4.32 -8.69 -0.87 -6.65  -1.8  ...  -3.54   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...    ...   
73415  None  None  None  None  5.78  8.45  None   8.2  None   8.5  ...   None   
73416  None  None  None  None  5.24  None  None  5.68  None  None  ...   None   
73417  None  None  None  None -3.93  None -9.03 -2.67  None  None  ...   None   
73418  None  None  None  None  9.42  9.66  None  9.17  None  None  ...   None   
73419  None  None  None  None  0.19  None  -1.5  7.82  None  None  ...   None   

      99.18 99.19 99.20 99.

C:\Users\Thomas Chourret\AppData\Local\Temp\ipykernel_832\3019423092.py:10: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dfp = dfp.fillna(mean)


          -7.82      8.79     -9.66     -8.16  -7.52      -8.5     -9.85  \
0      0.741947 -0.290000  6.360000  4.370000  -2.38 -9.660000 -0.730000   
1      0.741947  0.741947  0.741947  0.741947   9.03  9.270000  9.030000   
2      0.741947  8.350000  0.741947  0.741947   1.80  8.160000 -2.820000   
3      8.500000  4.610000 -4.170000 -5.390000   1.36  1.600000  7.040000   
4     -6.170000 -3.540000  0.440000 -8.500000  -7.09 -4.320000 -8.690000   
...         ...       ...       ...       ...    ...       ...       ...   
73415  0.741947  0.741947  0.741947  0.741947   5.78  8.450000  0.741947   
73416  0.741947  0.741947  0.741947  0.741947   5.24  0.741947  0.741947   
73417  0.741947  0.741947  0.741947  0.741947  -3.93  0.741947 -9.030000   
73418  0.741947  0.741947  0.741947  0.741947   9.42  9.660000  0.741947   
73419  0.741947  0.741947  0.741947  0.741947   0.19  0.741947 -1.500000   

       4.17     -8.98     -4.76  ...    2.82.2     99.18     99.19     99.20  \
0     -